In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

In [11]:
people = pd.read_csv('./people.csv')

En repartant du dataset "people.csv" (cf. lesson4) dans sa version "clean" finale:

   - mettre la colonne inactive à true pour tous les users dont le last_seen date d'au moins un an
   - avec une regex: filtrer les numéros de téléphone invalides
   - ajouter une colonne indiquant si le numéro de tel correspond à un téléphone portable (06/07)
   - ajouter une colonne indiquant si les coordonnées GPS de l'utilisateur correspondent bien au "country"



In [12]:

def clean_people(df):
    # rename columns:
    df = df.rename(columns={'email address': 'email'})
    
    # remove rows which have an empty "first_name" (NA):
    #df = df[df.first_name.notna()] <- equivalent to next line:
    df = df.dropna(subset=['first_name'])
    
    # drop duplicates on ID column:
    df = df.drop_duplicates()
    
    # Normalize gender column:
    df['gender'] = df['gender'].replace({'Female': 'F', 'Male': 'M'})
    
    # Convert column "age" to number (coerce: put NaN for bad values):
    df['age'] = pd.to_numeric(df.age, errors='coerce')
    
    # Convert columns to date type:
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    # When missing, last seen should fallback to the registration date:
    df['last_seen'] = df.last_seen.combine_first(df.registration)
    
    # Add a "full_name" column by concatenating two other ones:
    df['full_name'] = df.first_name + " " + df.last_name
    
    # Add a "country" column by extracting it from the address, with a split:
    df['country'] = df.address.str.split(', ').str[1]
    
    # Column "money" contains values like "$50.23" or "€23,09".
    # We want to make it uniform (only dollar currency) and as number, not str.
    df['currency'] = df.money.str[0]  # extract first char ($/€) to a new "currency" column
    df['money'] = df.money.str[1:].str.replace(',', '.')  # extract remaining chars and replace , by .
    df['money'] = pd.to_numeric(df.money)  # convert to number
    # convert euros cells to dollar:
    df.loc[df.currency == '€', 'money'] = df[df.currency == '€'].money * 1.10
    del df['currency']  # remove "currency" column which is now useless
    
    # Keep only rows where email is not NA:
    df = df.dropna(subset=['email'])
    # Keep only rows where email is a good email:
    # CAUTION: in the real world you should not use dummy regexes like this to validate email addresses,
    # but instead use a dedicated tool like https://github.com/syrusakbary/validate_email.
    df = df[df.email.str.contains('.+@[0-9a-zA-Z\.\-_]+\.\w{2,}')]
    # Some users may use email alias (example: john.smith+truc@gmail.com is an alias for john.smith@gmail.com).
    # We want to drop these duplicates. To do that, we extract the 'alias' part with a regex:
    groups = df.email.str.extract('([0-9a-zA-Z\.\-_]+)(\+[0-9a-zA-Z\.\-_]+)?(@[0-9a-zA-Z\.\-_]+\.\w{2,})')
    df['email'] = groups[0] + groups[2]  # we override the email with the email without the alias part
    # Then, just use drop_duplicates, which will keep the first line by default:
    df = df.drop_duplicates(subset=['email'])
    
    return df

df_clean = clean_people(people)



In [13]:
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,27465,Adelind,Christin,achristinz@blogs.com,F,30.0,90.024,41.427853,52.415968,NaN,2019-06-04,False,2019-06-04 00:00:00,"Znamenka, Russia",entrée/plat/dessert,Adelind Christin,Russia
205,27476,Petronella,Pickance,ppickance1a@uiuc.edu,F,28.0,40.865,132.702111,33.762296,NaN,2017-11-22,False,2018-07-29 13:18:29,"Iyo, Japan",entrée/plat/dessert,Petronella Pickance,Japan
206,27521,Moritz,Issacof,missacof2j@wired.com,M,21.0,91.970,72.311099,31.263396,0298949280,2018-08-20,False,2019-02-10 16:39:03,"Jhang Sadr, Pakistan",entrée/plat/dessert,Moritz Issacof,Pakistan
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32.0,61.204,111.138442,-6.783653,0734595126,2018-10-25,False,2018-10-25 00:00:00,"Mantingantengah, Indonesia",entrée/plat/dessert,Quintana Foulstone,Indonesia


In [14]:
#mettre la colonne inactive à true pour tous les users dont le last_seen date d'au moins un an
df_clean['inactive'] = df_clean['last_seen'] < datetime.strptime('2018-10-23', '%Y-%m-%d')


In [15]:
df_clean = df_clean.dropna(subset = ['phone'])

In [20]:
#avec une regex: filtrer les numéros de téléphone invalides
df_clean = df_clean[df_clean['phone'].apply(str).str.contains('^0\d{9}$')]

In [21]:
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country,Mobile,isPhone
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic,True,True
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic,True,True
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China,True,True
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.500,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia,True,True
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,27624,Dur,Hallaways,dhallaways5e@delicious.com,M,25.0,76.076,106.720976,28.739865,0317009357,2019-07-27,False,2019-09-23 05:01:36,"Ganshui, China",entrée/plat/dessert,Dur Hallaways,China,True,True
199,27604,Michelina,Lowcock,mlowcock4u@nba.com,F,42.0,NaN,12.325710,58.335399,0276520906,2017-07-12,True,2017-07-12 00:00:00,"Vänersborg, Sweden",entrée/plat/dessert,Michelina Lowcock,Sweden,True,True
206,27521,Moritz,Issacof,missacof2j@wired.com,M,21.0,91.970,72.311099,31.263396,0298949280,2018-08-20,False,2019-02-10 16:39:03,"Jhang Sadr, Pakistan",entrée/plat/dessert,Moritz Issacof,Pakistan,True,True
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32.0,61.204,111.138442,-6.783653,0734595126,2018-10-25,False,2018-10-25 00:00:00,"Mantingantengah, Indonesia",entrée/plat/dessert,Quintana Foulstone,Indonesia,True,True


In [22]:
#- ajouter une colonne indiquant si le numéro de tel correspond à un téléphone portable (06/07)
def isMobile(x):
    if x[:2] == "06":
        return True
    elif x[:2] == "07":
        return True
    else:
        return False

In [23]:
df_clean['isMobile']=df_clean['phone'].apply(isMobile)

/home/yamhiroto/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country,Mobile,isPhone,isMobile
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic,True,True,False
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic,True,True,False
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China,True,True,False
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.500,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia,True,True,False
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,27624,Dur,Hallaways,dhallaways5e@delicious.com,M,25.0,76.076,106.720976,28.739865,0317009357,2019-07-27,False,2019-09-23 05:01:36,"Ganshui, China",entrée/plat/dessert,Dur Hallaways,China,True,True,False
199,27604,Michelina,Lowcock,mlowcock4u@nba.com,F,42.0,NaN,12.325710,58.335399,0276520906,2017-07-12,True,2017-07-12 00:00:00,"Vänersborg, Sweden",entrée/plat/dessert,Michelina Lowcock,Sweden,True,True,False
206,27521,Moritz,Issacof,missacof2j@wired.com,M,21.0,91.970,72.311099,31.263396,0298949280,2018-08-20,False,2019-02-10 16:39:03,"Jhang Sadr, Pakistan",entrée/plat/dessert,Moritz Issacof,Pakistan,True,True,False
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32.0,61.204,111.138442,-6.783653,0734595126,2018-10-25,False,2018-10-25 00:00:00,"Mantingantengah, Indonesia",entrée/plat/dessert,Quintana Foulstone,Indonesia,True,True,True


In [ ]:
# - ajouter une colonne indiquant si les coordonnées GPS de l'utilisateur correspondent bien au "country"

